In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error


In [3]:
!pip install pyarrow -q

In [4]:
categorical = ['PULocationID', 'DOLocationID']
numerical = []
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    print('shape:', df.shape)
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    print('duration std before filtering outliers', df['duration'].std())
    count_before_filter = len(df)
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    count_after_filter = len(df)
    print('% left after filtering by time', count_after_filter/count_before_filter*100)
    df[categorical] = df[categorical].astype(str)
    return df

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

shape: (3066766, 19)
duration std before filtering outliers 42.594351241920904
% left after filtering by time 98.1220282212598


In [6]:
Q1: 19
Q2: 42.59
Q3: 98

In [7]:
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

shape: (2913955, 19)
duration std before filtering outliers 42.84210176105097
% left after filtering by time 98.00944077722545


In [8]:
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
print('Matrix size:', X_train.shape)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


Matrix size: (3009173, 515)


Q4: 515

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [11]:
RMSE = mean_squared_error(y_train, lr.predict(X_train), squared=False)
print(RMSE)

7.649261027826866


In [12]:
Q5: 7.64

In [13]:
y_pred = lr.predict(X_val)

In [14]:
RMSE = mean_squared_error(y_val, y_pred, squared=False)

In [15]:
RMSE

7.811832641626525

Q6: 7.81